<a href="https://colab.research.google.com/github/AhmedAlshoki/NewYork-City-Taxi-fare-Predictions/blob/main/NewWorkTaxiFarePredictionJovian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.set_project('newworktaxifareprediction')
jovian.set_colab_id('1ph14n_sqR0CTvrr7occdY7JUmfw5-WUS')

In [ ]:
#!pip install opendatasets scikit-learn 

In [ ]:
#data_url = 'https://www.kaggle.com/competitions/new-york-city-taxi-fare-prediction'
#od.download(data_url)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
#import opendatasets as od
from tqdm import tqdm
import jovian as jovian

In [ ]:
data_dir = '/content/drive/MyDrive/Kaggle/new-york-city-taxi-fare-prediction'

In [ ]:
selected_cols= 'key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count'.split(',')
dtypes = {
 'fare_amount':'float32',
 'pickup_longitude':'float32',
 'pickup_latitude':'float32',
 'dropoff_longitude':'float32',
 'dropoff_latitude':'float32',
 'passenger_count':'uint8'
}

In [ ]:
def coerce(x,min,max):
    if x <= max:
        return max
    elif x >= min:
        return min

In [ ]:
def Calculate_Distance(x1,y1,x2,y2):
   Horizontal_Distance = x2-x1
   Vertical_Distance = y2-y1
   Square_distace = (Horizontal_Distance*Horizontal_Distance)+(Vertical_Distance*Vertical_Distance)
   
   return Square_distace

In [ ]:
def prepare_train_set(train_set):
   train_set['passenger_count']  = train_set['passenger_count'].fillna(np.mean(train_set['passenger_count']) if np.mean(train_set['passenger_count'])!=None else 0)
   train_set['passenger_count']= train_set.apply(lambda row: coerce(row['passenger_count'],1,6), axis=1)
   #============
   train_set['pickup_longitude']  = train_set['pickup_longitude'].fillna(np.mean(train_set['pickup_longitude']) if np.mean(train_set['pickup_longitude'])!=None else 0)
   train_set['pickup_longitude']= train_set.apply(lambda row: coerce(row['pickup_longitude'],-75.0,-72.0), axis=1)
   #===========
   train_set['pickup_latitude']  = train_set['pickup_latitude'].fillna(np.mean(train_set['pickup_latitude']) if np.mean(train_set['pickup_latitude'])!=None else 0)
   train_set['pickup_latitude']= train_set.apply(lambda row: coerce(row['pickup_latitude'],39.0,40.0), axis=1)
   #========
   train_set['dropoff_longitude']  = train_set['dropoff_longitude'].fillna(np.mean(train_set['dropoff_longitude']) if np.mean(train_set['dropoff_longitude'])!=None else 0)
   train_set['dropoff_longitude']= train_set.apply(lambda row: coerce(row['dropoff_longitude'],-75.0,-72.0), axis=1)
   #===========
   train_set['dropoff_latitude']  = train_set['dropoff_latitude'].fillna(np.mean(train_set['dropoff_latitude']) if np.mean(train_set['dropoff_latitude'])!=None else 0)
   train_set['dropoff_latitude']= train_set.apply(lambda row: coerce(row['dropoff_latitude'],39.0,40.0), axis=1)
   try :
    train_set['fare_amount'] = detect_outliers(train_set['fare_amount'])
    train_set['fare_amount']= train_set.apply(lambda row: coerce(row['fare_amount'],1.0,500.0), axis=1)
   except:
    pass
   #============================================================================
   train_set['Distance of the trip'] = Calculate_Distance(train_set['pickup_longitude'],train_set['pickup_latitude'],
                                                          train_set['dropoff_longitude'],train_set['dropoff_latitude'])
   train_set.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1, inplace=True)
   #=======================================================
   train_set['pickup_datetime Year'] = pd.DatetimeIndex(train_set['pickup_datetime']).year
   train_set['pickup_datetime Year']  = train_set['pickup_datetime Year'].fillna(np.mean(train_set['pickup_datetime Year']))
   train_set['pickup_datetime Month'] = pd.DatetimeIndex(train_set['pickup_datetime']).month
   train_set['pickup_datetime Month']  = train_set['pickup_datetime Month'].fillna(np.mean(train_set['pickup_datetime Month']))
   train_set['pickup_datetime Day'] = pd.DatetimeIndex(train_set['pickup_datetime']).day
   train_set['pickup_datetime Day']  = train_set['pickup_datetime Day'].fillna(np.mean(train_set['pickup_datetime Day']))
   train_set['pickup_datetime Hour'] = pd.DatetimeIndex(train_set['pickup_datetime']).hour
   train_set['pickup_datetime Hour']  = train_set['pickup_datetime Hour'].fillna(np.mean(train_set['pickup_datetime Hour']))
   train_set['pickup_datetime Mintues'] = pd.DatetimeIndex(train_set['pickup_datetime']).minute
   train_set['pickup_datetime Mintues']  = train_set['pickup_datetime Mintues'].fillna(np.mean(train_set['pickup_datetime Mintues']))
   train_set.drop('pickup_datetime',axis=1, inplace=True) 
   return train_set

In [ ]:
from tqdm import tqdm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
import pickle

In [ ]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ··········
[jovian] Uploading colab notebook to Jovian...
Committed successfully! https://jovian.ai/ahmedalshoki/newworktaxifareprediction


'https://jovian.ai/ahmedalshoki/newworktaxifareprediction'

Train Model

In [ ]:
for i in range (0,56):
    train_set_chunk = pd.read_csv(data_dir+'/train.csv',nrows=1000000,skiprows=100000*i,parse_dates=[2])
    train_set_chunk.columns = selected_cols
    train_set_chunk = prepare_train_set(train_set_chunk)
    target = train_set_chunk.pop('fare_amount')
    train_set_chunk.drop('key',axis=1, inplace=True) 
    poly = PolynomialFeatures(degree=3, include_bias=False)
    print(i)
    poly_features = poly.fit_transform(train_set_chunk)
    try:
        model = pickle.load(open(data_dir+'/PolynomialRegression.sav', "rb"))
        model.fit(poly_features,np.array(target))
        filename_path = data_dir+'/PolynomialRegression.sav'
        pickle.dump(model, open(filename_path, 'wb')) 
    except:  
        model = LinearRegression()
        model.fit(poly_features,np.array(target))
        filename_path = data_dir+'/PolynomialRegression.sav'
        pickle.dump(model, open(filename_path, 'wb'))    
    del train_set_chunk    

In [ ]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
Committed successfully! https://jovian.ai/ahmedalshoki/newworktaxifareprediction


'https://jovian.ai/ahmedalshoki/newworktaxifareprediction'

In [ ]:
test_set = pd.read_csv(data_dir+'/test.csv',parse_dates=['pickup_datetime'])
sample_submission = pd.read_csv(data_dir+'/sample_submission.csv')

In [ ]:
sample_submission

In [ ]:
test_set

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44+00:00,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12+00:00,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12+00:00,-73.966046,40.789775,-73.988565,40.744427,1
...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,2015-05-10 12:37:51+00:00,-73.968124,40.796997,-73.955643,40.780388,6
9910,2015-01-12 17:05:51.0000001,2015-01-12 17:05:51+00:00,-73.945511,40.803600,-73.960213,40.776371,6
9911,2015-04-19 20:44:15.0000001,2015-04-19 20:44:15+00:00,-73.991600,40.726608,-73.789742,40.647011,6
9912,2015-01-31 01:05:19.0000005,2015-01-31 01:05:19+00:00,-73.985573,40.735432,-73.939178,40.801731,6


In [ ]:
model = pickle.load(open(data_dir+'/PolynomialRegression.sav', "rb"))
keys = test_set.pop('key')
test_set = prepare_train_set(test_set)
poly = PolynomialFeatures(degree=3, include_bias=False)
poly_features = poly.fit_transform(test_set)
test_predection = model.predict(np.array(poly_features))
test_predection = np.array(test_predection).reshape(-1,1)
test_predection = test_predection.ravel()
test_pred = pd.Series(test_predection)
test_set['fare_amount'] = test_pred
test_set['key'] = keys

In [ ]:
sample_submission['fare_amount_prediction'] = test_pred

In [ ]:
sample_submission.describe()

,fare_amount,fare_amount_prediction
count,9.914000e+03,9914.000000
mean,1.135000e+01,11.395892
std,3.552893e-15,1.246870
min,1.135000e+01,8.956850
25%,1.135000e+01,10.322251
50%,1.135000e+01,10.960687
75%,1.135000e+01,12.663439
max,1.135000e+01,13.805783


In [ ]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: 